In [142]:
import numpy as np
from utils import read_img, write_img


def padding(img, padding_size, type):
    """
        The function you need to implement for Q1 a).
        Inputs:
            img: array(float)
            padding_size: int
            type: str, zeroPadding/replicatePadding
        Outputs:
            padding_img: array(float)
    """

    x, y = img.shape
    padding_img = np.zeros((x + 2 * padding_size, y + 2 * padding_size))
    padding_img[padding_size:x + padding_size,
                padding_size:y + padding_size] = img
    if type == "zeroPadding":
        return padding_img
    elif type == "replicatePadding":
        padding_img[0:padding_size, 0:padding_size] = img[0][0]
        padding_img[x + padding_size:x + 2 * padding_size,
                    0:padding_size] = img[x - 1][0]
        padding_img[0:padding_size,
                    y + padding_size:y + 2 * padding_size] = img[0][y - 1]
        padding_img[x + padding_size:x + 2 * padding_size,
                    y + padding_size:y + 2 * padding_size] = img[x - 1][y - 1]
        padding_img[0:padding_size, padding_size:y + padding_size] = img[0,
                                                                         0:y]
        padding_img[x + padding_size:x + 2 * padding_size,
                    padding_size:y + padding_size] = img[x - 1, 0:y]
        padding_img[padding_size:x + padding_size,
                    0:padding_size] = img[0:x, 0].reshape(x, 1)
        padding_img[padding_size:x + padding_size, y + padding_size:y +
                    2 * padding_size] = img[0:x, y - 1].reshape(x, 1)
        return padding_img

def convol_with_Toeplitz_matrix_1(img, kernel):
    """
        The function you need to implement for Q1 b).
        Inputs:
            img: array(float) 6*6
            kernel: array(float) 3*3
        Outputs:
            output: array(float)
    """
    #zero padding
    padding_img = padding(img, 1, "zeroPadding")

    #build the Toeplitz matrix and compute convolution
    T = np.zeros((36, 64))
    idx = np.reshape(np.reshape(np.arange(6) * 8, (6,1)) + (np.arange(6)), (36,))
    # idx = np.concatenate((idx, (idx + 8), (idx + 16), (idx + 24), (idx + 32), (idx + 40)), axis = None)
    
    sft = np.reshape(np.arange(3) * 8, (3,1)) + (np.arange(3))
    idx = np.reshape(idx, (36, 1, 1)) + sft
    T[np.broadcast_to(np.reshape(np.arange(36), (36, 1, 1)), (36, 3, 3)), idx] += kernel
    # for i in range(3):
    #     for j in range(3):
    #         T[np.arange(36), idx + j + i*8] += kernel[i,j]
    
    return np.reshape(np.matmul(T, np.reshape(padding_img, (64,1))), (6,6))
def convol_with_Toeplitz_matrix(img:np.ndarray, kernel:np.ndarray):
    """
        The function you need to implement for Q1 b).
        Inputs:
            img: array(float) 6*6
            kernel: array(float) 3*3
        Outputs:
            output: array(float)
    """
    #zero padding
    padding_img:np.ndarray = padding(img,1,"zeroPadding").reshape(64,1)

    #build the Toeplitz matrix and compute convolution
    kernel=kernel.reshape(9)
    k1=list(kernel[0:3])
    k2=list(kernel[3:6])
    k3=list(kernel[6:9])
    l1=k1+[0]*5+k2+[0]*5+k3+[0]*46 # 3+5+3+5+3+46=65=64+1(循环往右移一位)
    l2=l1*6+[0]*2 # 补足 2 位，分块
    l3=l2*5+l1*5+k1+[0]*5+k2+[0]*5+k3 # 处理最后一行
    topl_mat=np.array(l3).reshape(36,64)
    output=(topl_mat@padding_img).reshape(6,6)
    return output
np.set_printoptions(threshold=np.inf)
img=np.array([[0]*6,
              [1]*6,
              [2]*6,
              [3]*6,
              [4]*6,
              [5]*6
              ])

kernel=np.array([[1,2,3],[4,5,6],[7,8,9]])
print(img)
print(kernel)
convol_with_Toeplitz_matrix_1(img,kernel)



[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 1. 1. 1. 0.]
 [0. 2. 2. 2. 2. 2. 2. 0.]
 [0. 3. 3. 3. 3. 3. 3. 0.]
 [0. 4. 4. 4. 4. 4. 4. 0.]
 [0. 5. 5. 5. 5. 5. 5. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]
[[1 2 3]
 [4 5 6]
 [7 8 9]]


array([[ 17.,  24.,  24.,  24.,  24.,  15.],
       [ 45.,  63.,  63.,  63.,  63.,  39.],
       [ 78., 108., 108., 108., 108.,  66.],
       [111., 153., 153., 153., 153.,  93.],
       [144., 198., 198., 198., 198., 120.],
       [ 75.,  99.,  99.,  99.,  99.,  57.]])

In [134]:
def convolve_e(img, kernel):
    """
        The function you need to implement for Q1 c).
        Inputs:
            img: array(float)
            kernel: array(float)
        Outputs:
            output: array(float)
    """
    
    #build the sliding-window convolution here
    n,m = img.shape
    k,l = kernel.shape
    H = np.broadcast_to(np.reshape(np.arange(n-k+1), (n-k+1,1,1,1)), (n-k+1,m-l+1,1,1)) + np.broadcast_to(np.reshape(np.arange(k), (k, 1)), (k, l))
    L = np.broadcast_to(np.reshape(np.arange(m-l+1), (1,m-l+1,1,1)), (n-k+1,m-l+1,1,1)) + np.broadcast_to(np.reshape(np.arange(l), (1, l)), (k, l))
    T = img[H, L]
    output = np.sum(np.sum(T * kernel, axis=3), axis=2)
    return output


In [146]:
def convolve(img, kernel):
    """
        The function you need to implement for Q1 c).
        Inputs:
            img: array(float)
            kernel: array(float)
        Outputs:
            output: array(float)
    """
    
    #build the sliding-window convolution here
    
    image_height, image_width = img.shape
    kernel_height, kernel_width = kernel.shape
    output_height = image_height - kernel_height + 1
    output_width = image_width - kernel_width + 1
    
    output = np.zeros((output_height, output_width))

    kernel_i,kernel_j=np.meshgrid(np.arange(kernel_height),np.arange(kernel_width),indexing="ij")
    kernel_i=kernel_i.reshape(1,kernel_height*kernel_width)
    kernel_j=kernel_j.reshape(1,kernel_height*kernel_width)
    kernel_i-=1
    kernel_j-=1
    
    img_i,img_j=np.meshgrid(np.arange(image_height)[kernel_height//2:image_height-kernel_height//2],
                            np.arange(image_width)[kernel_width//2:image_width-kernel_width//2],indexing="ij")
    
    img_i=img_i.reshape((output_width*output_height,1))
    img_i=np.repeat(img_i,kernel_height*kernel_width).\
        reshape((output_width*output_height,kernel_height*kernel_width))
    
    img_j=img_j.reshape((output_width*output_height,1))
    img_j=np.repeat(img_j,kernel_height*kernel_width).\
        reshape((output_width*output_height,kernel_height*kernel_width))
    
    img_i+=kernel_i
    img_j+=kernel_j

    
    img_sliding_window=img[img_i,img_j]
    kernel=kernel.reshape(1,kernel_height*kernel_width)
    kernel=np.repeat(kernel,output_width*output_height,axis=0).\
        reshape((output_width*output_height,kernel_height*kernel_width))
    
    output=np.sum(img_sliding_window*kernel,axis=1).reshape((output_height,output_width))
    return output
convolve(padding(img,1,"zeroPadding"),kernel)

array([[ 17.,  24.,  24.,  24.,  24.,  15.],
       [ 45.,  63.,  63.,  63.,  63.,  39.],
       [ 78., 108., 108., 108., 108.,  66.],
       [111., 153., 153., 153., 153.,  93.],
       [144., 198., 198., 198., 198., 120.],
       [ 75.,  99.,  99.,  99.,  99.,  57.]])

In [136]:
a=np.array([[[1,2,3],[4,5,6],[7,8,9]],
            [[0,2,3],[4,5,6],[7,8,9]]])
b=np.array([[1,2,3],[4,5,6],[7,8,9]])
convolve_e(padding(img,1,"zeroPadding"),kernel)


array([[ 17.,  24.,  24.,  24.,  24.,  15.],
       [ 45.,  63.,  63.,  63.,  63.,  39.],
       [ 78., 108., 108., 108., 108.,  66.],
       [111., 153., 153., 153., 153.,  93.],
       [144., 198., 198., 198., 198., 120.],
       [ 75.,  99.,  99.,  99.,  99.,  57.]])

In [118]:
np.repeat([[1,2,3]],9,axis=0)

array([[1, 2, 3],
       [1, 2, 3],
       [1, 2, 3],
       [1, 2, 3],
       [1, 2, 3],
       [1, 2, 3],
       [1, 2, 3],
       [1, 2, 3],
       [1, 2, 3]])

In [148]:
np.append(np.array([1,2,3]),np.array([3,4,5]))

array([1, 2, 3, 3, 4, 5])

In [149]:
(1,2)+(3,4)

(1, 2, 3, 4)